<b>Загрузим нужные библиотеки</b>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
df = pd.read_csv("/content/train.csv")

<b>Рассмотрим датасет по ближе</b>

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df["Категория"].hist(bins = 17)

Заметен сильный разброс популярности категориий

In [ ]:
df["Ответственное лицо"].value_counts()

Сделаем категорию "Ответственное лицо" раноговой

In [ ]:
df["Ответственное лицо"] = pd.Categorical(df["Ответственное лицо"])
df["Ответственное лицо"].astype('category').cat.codes
df["Ответственное лицо"] = df["Ответственное лицо"].cat.codes

In [ ]:
sns.countplot(x = "Ответственное лицо" , data  = df)

In [ ]:
sns.jointplot(x = "Ответственное лицо", y = 'Категория', data = df, kind = 'reg')

Кажется нет четкой зависимости между категорией и ответственным лицом города

In [2]:
df["Тематика"] = pd.Categorical(df["Тематика"])
df["Тематика"].astype('category').cat.codes
df["Тематика"] = df["Тематика"].cat.codes

<class 'NameError'>: name 'pd' is not defined

In [ ]:
Видоизменим категорию "Тематика"

In [ ]:
df["Длина Сообщения"] = df["Текст Сообщения"].str.len()

Добавим новый столбец в датсет и взглянем на то что у нас получилось

In [ ]:
df.head(3)

In [ ]:
plt.rcParams['figure.figsize']=(15,15)

corr = df.corr()
g = sns.heatmap(corr, square = True, annot=True)

<b>Последняя попытка найти явную зависимость в данных<b><br>
<b>Выделим выборки</b>

In [ ]:
X = df.drop(["Категория", "Текст Сообщения", "id"], axis = 1)
y = df[["Категория"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

<b>Обучение модели</b>

In [ ]:
clf = RandomForestClassifier(random_state=0)

In [ ]:
clf.fit(X_train, y_train)

<b>Оценка точности</b>

In [ ]:
pred = clf.predict(X_test)

In [ ]:
def one_label_to_many(df, classes = [x for x in range(17)]):
    y_test = []
    min_class = min(classes)
    count_class = len(classes)

    for ll in df:
        mass = [0]*count_class
        mass[int(ll) - min_class] = 1
        y_test.append(mass)

    return y_test

In [ ]:
y_test["Категория"].value_counts()

In [ ]:
from sklearn.metrics import roc_auc_score

pred_new = one_label_to_many(pred)
y_test["Категория"].iloc[0] = 12
y_test["Категория"].iloc[2] = 14 


score = roc_auc_score(y_test["Категория"], pred_new , multi_class='ovo')

In [ ]:
score